In [1]:
import time
import numpy as np

class Game:
    def __init__(self):
        self.initialize_game()

    def initialize_game(self):
        self.current_state = [['.','.','.'],
                              ['.','.','.'],
                              ['.','.','.']]

        # Giocatore che gioca per primo
        self.player_turn = 'X'

    def draw_board(self):
        for i in range(0, 3):
            for j in range(0, 3):
                print('{}|'.format(self.current_state[i][j]), end=" ")
            print()
        print()
        
    # Verifica se la mossa giocata è legale
    def is_valid(self, px, py):
        if px < 0 or px > 2 or py < 0 or py > 2:
            return False
        elif self.current_state[px][py] != '.':
            return False
        else:
            return True
    
    # Verifica se lo stato è terminale, e in tal caso restituisce il vincitore
    def is_end(self):
        
        # Vittoria su una colonna
        for i in range(0, 3):
            if (self.current_state[0][i] != '.' and
                self.current_state[0][i] == self.current_state[1][i] and
                self.current_state[1][i] == self.current_state[2][i]):
                return self.current_state[0][i]

        # Vittoria su una riga
        for i in range(0, 3):
            if (self.current_state[i] == ['X', 'X', 'X']):
                return 'X'
            elif (self.current_state[i] == ['O', 'O', 'O']):
                return 'O'

        # Vittoria sulla diagonale principale
        if (self.current_state[0][0] != '.' and
            self.current_state[0][0] == self.current_state[1][1] and
            self.current_state[0][0] == self.current_state[2][2]):
            return self.current_state[0][0]

        # Vittoria sulla diagonale secondaria
        if (self.current_state[0][2] != '.' and
            self.current_state[0][2] == self.current_state[1][1] and
            self.current_state[0][2] == self.current_state[2][0]):
            return self.current_state[0][2]

        # La scacchiera è piena?
        for i in range(0, 3):
            for j in range(0, 3):
                # C'è una casella vuota; il gioco può continuare.
                if (self.current_state[i][j] == '.'):
                    return None
        # E' patta!
        return 'patta'
    
    
    # Il giocatore 'O' è max, in questo caso AI
    def max(self):

        # I possibili valori per maxv sono:
        # -1 - sconfitta
        # 0  - patta
        # 1  - vittoria

        # Inizialmente impostiamo maxv a un low value:
        maxv = -np.inf

        px = None
        py = None

        result = self.is_end()

        # Se siamo arrivati a uno stato terminale, il metodo deve restituire
        # il valore della "funzione di utilità"  dello stato. 
        # Essa può essere:
        # -1 - sconfitta
        # 0  - patta
        # 1  - vittoria
        
        if result == 'X':        # ha vinto il giocatore X
            return (-1, 0, 0)
        elif result == 'O':      # ha vinto il giocatore O
            return (1, 0, 0)
        elif result == 'patta':  # la partita è patta
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.current_state[i][j] == '.':
                    # Nella casella vuota il giocatore 'O' fa una mossa e chiama min
                    self.current_state[i][j] = 'O'
                    (m, min_i, min_j) = self.min() 
                    
                    # aggiorniamo il valore minimax corrente
                    # e la mossa corrispondente se necessario
                    if m > maxv:
                        maxv = m
                        px = i
                        py = j
                        
                    # Impostiamo di nuovo la casella a vuota.
                    self.current_state[i][j] = '.'
        return (maxv, px, py)
    
    
    # Il giocatore 'X' è min, in questo caso l'utente
    def min(self):

        # I possibili valori per minv sono:
        # -1 - vittoria
        # 0  - patta
        # 1  - sconfitta

        # Inizialmente impostiamo minv a un high value:
        minv = np.inf

        qx = None
        qy = None

        result = self.is_end()
        
        # Se siamo arrivati a uno stato terminale, il metodo deve restituire
        # il valore della "funzione di utilità"  dello stato. 
        # Essa può essere:
        # -1 - vittoria
        # 0  - patta
        # 1  - sconfitta

        if result == 'X':        # ha vinto il giocatore X
            return (-1, 0, 0)
        elif result == 'O':      # ha vinto il giocatore O
            return (1, 0, 0)
        elif result == 'patta':  # la partita è patta
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.current_state[i][j] == '.':
                    # Nella casella vuota il giocatore 'X' fa una mossa e chiama max
                    self.current_state[i][j] = 'X'
                    (m, max_i, max_j) = self.max()
                    
                    # aggiorniamo il valore minimax corrente
                    # e la mossa corrispondente se necessario
                    if m < minv:
                        minv = m
                        qx = i
                        qy = j
                    self.current_state[i][j] = '.'

        return (minv, qx, qy)
    
    
    def play(self):
        while True:
            self.draw_board()
            self.result = self.is_end()

            # Stampa dell'appropriato messaggio se il gioco è finito
            if self.result != None:
                if self.result == 'X':
                    print('Il vincitore è X!')
                elif self.result == 'O':
                    print('Il vincitore è O!')
                elif self.result == 'patta':
                    print('La partita è patta!')
                return

            # E' il turno dell'utente
            if self.player_turn == 'X':
                while True:

                    start = time.time()
                    (m, qx, qy) = self.min()
                    end = time.time()
                    print('Evaluation time: {}s'.format(round(end - start, 7)))
                    print('Mossa raccomandata: X = {}, Y = {}'.format(qx, qy))

                    px = int(input('Inserisci l\'indice di riga: '))
                    py = int(input('Inserisci l\'indice di colonna: '))

                    if self.is_valid(px, py):
                        self.current_state[px][py] = 'X'
                        self.player_turn = 'O'
                        break
                    else:
                        print('La mossa non è valida! Prova ancora.')

            # E' il turno di AI
            else:
                (m, px, py) = self.max()
                self.current_state[px][py] = 'O'
                self.player_turn = 'X'

In [ ]:
g = Game()
g.play()